In [13]:
! conda install html5lib

Solving environment: done

# All requested packages already installed.



In [23]:
import numpy as np 
import pandas as pd
import json 
import requests 
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import lxml

#STEP_ONE: get data from the wiki url
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
# use BeautifulSoup to open it, define the referenced source list as canada_list
canada_list = BeautifulSoup(source, 'lxml') 
#creat the target dataframe columns and give them a name
toronto_list = pd.DataFrame(columns=['Postalcode','Borough', 'Neighborhood'])

#STEP_TWO: The most import loop process to get data from the web
#inspect structure: div(class)->table(class)->  {thead()->tr()->th(class)} OR {tbody()->tr()->td(table elements)}
content = canada_list.find('div', class_='mw-parser-output')
table = content.table.tbody #can't be writen as the same row from above row
postcode = 0
borough = 0
neighborhood = 0

#loop each elements into the table, which located at {tbody()->tr()->td(table elements)}
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:  #the first element followed by <td> as postcode
            postcode = td.text
            i = i + 1
        elif i == 1: #the second element followed by <td> as borough
            borough = td.text
            i = i + 1
        elif i == 2: #the third element followed by <td> as neighborhood
            neighborhood = td.text.strip('\n').replace(']','') #need to gather neighbors in one cell
    toronto_list = toronto_list.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

#STEP_THREE: Remove unwanted values
# remove Not assigned borough, both 'not assigned or = 0'
toronto_list = toronto_list[toronto_list.Borough!='Not assigned']
toronto_list = toronto_list[toronto_list.Borough!= 0]
toronto_list.reset_index(drop = True, inplace = True)

#
i = 0
for i in range(0,toronto_list.shape[0]):
    if toronto_list.iloc[i][2] == 'Not assigned':
        toronto_list.iloc[i][2] = toronto_list.iloc[i][1]
        i = i+1
        
#print the dataframe
df = toronto_list.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [24]:
df.shape


(180, 3)

In [37]:

#clean postalcode and meerge datasets
df_coord = pd.read_csv("http://cocl.us/Geospatial_data")
df_coord.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
df_coord['Postalcode'] = df_coord['Postalcode'].astype(str)
df_coord['Postalcode'] = df_coord['Postalcode'].str.strip()
df['Postalcode'] = df['Postalcode'].astype(str)
df['Postalcode'] = df['Postalcode'].str.strip()
df1 = pd.merge(df, df_coord, how="inner", on = 'Postalcode')
df1.head(12)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough\n,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough\n,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough\n,Woburn,43.770992,-79.216917
4,M1H,Scarborough\n,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough\n,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough\n,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough\n,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [17]:
CLIENT_ID = 'AST4DMTUZUNTVYOYXEHJ3PAKUT4CKQQFU03FME25GF3YCMK2' # your Foursquare ID
CLIENT_SECRET = 'TUOQ1SWN0EKEHFGJ3KCUMSVEE05KCIBR3AJZ2XHSDHBACXMZ' # your Foursquare Secret
VERSION = '20190330' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AST4DMTUZUNTVYOYXEHJ3PAKUT4CKQQFU03FME25GF3YCMK2
CLIENT_SECRET:TUOQ1SWN0EKEHFGJ3KCUMSVEE05KCIBR3AJZ2XHSDHBACXMZ
